# Practical 7 - Part 2A
This second half of the lab explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

In this section, we'll work on building two components that we need for 2B, a method to estimate that transformation and a method that can project points into camera image space.

First we'll tackle the projection method, `projectiveCamera`. We want to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that takes us from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`. Essentially we want to compute the extrinsics matrix we can use in `projectiveCamera`.

Estimating the camera pose will involve calculating a homography, so you'll need to copy over your functions from part 1A/1B in the space provided.

Enric Balaguer Rodon, 
Student Number: 23058643

## Import libraries 

In [57]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [58]:
def projectiveCamera(K,T,XCart):
    ##TODO
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points    
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    # TO DO: Replace this
    real_world_cartesian = XCart

    # TO DO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    real_world_homogenous = np.vstack((real_world_cartesian, np.ones(real_world_cartesian.shape[1])))

    # TO DO: Apply extrinsic matrix to XHom, to move to frame of reference of camera
    camera_space_homogenous = np.matmul(T, real_world_homogenous)
    
    # TO DO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    camera_space_homogenous_normalised = np.delete(camera_space_homogenous, -1, axis=0)

    # TO DO: Move points to image coordinates xImHom by applying intrinsic matrix
    image_space_homogenous = np.matmul(K, camera_space_homogenous_normalised)

    # TO DO: Convert points back to Cartesian coordinates xImCart
    image_space_cartesian = image_space_homogenous[0:2,:]/np.tile(image_space_homogenous[2,:],(2,1))

    return image_space_cartesian


## Camera Projection

First we'll write up the function that can take us from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

To verify that your solution is correct please compare your image space points to those in the comment.

Once they match, move on to the next bit - estimating a transformation! 

In [59]:
# We assume that the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# TO DO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera"
XImCart = projectiveCamera(K,T,XCart)

print(XImCart)
# Should be around:
# [267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
# [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]

[[267.417 230.95  531.425 482.36  378.775]
 [396.268 288.114 237.834 358.399 329.441]]


In [60]:
print("T.shape =", T.shape)
print("K.shape =", K.shape)
print("real_world_cartesian.shape =", XCart.shape)

T.shape = (4, 4)
K.shape = (3, 3)
real_world_cartesian.shape = (3, 5)


### You've implemented both of these functions in 1A and 1B already, so feel free to copy them in here. You'll need them for this next part.

In [61]:
def solveAXEqualsZero(A):
    # TO DO: Write this routine - it should solve Ah = 0. You can do this using SVD. Consult your notes! 
    # Hint: SVD will be involved. 
    U, L, V_transposed = np.linalg.svd(A) #A = ULV^T
    #Now acquire last column of V, and that's h
    last_column_V = np.transpose(V_transposed)[:,-1]
    h = last_column_V
    return h

In [62]:
def calcBestHomography(pts1Cart, pts2Cart):
    
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian points 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    # Get number of images
    N = pts1Cart.shape[1]

    # Get homogenous representation of real world points
    real_world_homogenous = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    screen_images = pts2Cart
    #Construct A
    #A = np.zeros(N*2,9)
    A = []
    for n in range(N):
        rwh = real_world_homogenous[:,n]
        sh = screen_images[:,n]

        #Get first row to add per datapoint from the real world
        dummy_list = [0, -1, sh[1]]
        row1 = []
        for variable in dummy_list:
            for value in rwh:
                row1.append(variable*value)

        #Get second row to add per datapoint from the real world
        dummy_list = [1, 0, -sh[0]]
        row2 = []
        for variable in dummy_list:
            for value in rwh:
                row2.append(variable*value)

        A.append(row1)
        A.append(row2)
    
    A = np.array(A)
    
    # We now solve for h as defined above
    H = solveAXEqualsZero(A)

    # Reshape as desired
    H = H.reshape(3,3)

    # TO DO: replace this:
    # TO DO: 
    # First convert points into homogeneous representation
    # Hint: we've done this before  in the skeleton code we provide.
    
    # Then construct the matrix A, size (n_points * 2, 9)
    # Consult the notes!
    
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    
    # Reshape h into the matrix H, values of h go first into rows of H

    
    return H

In [63]:
# Read the next cell first for context!

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.
    
    # TO DO: replace this

    real_world_cartesian = XCart
    image_space_cartesian = XImCart
    # TO DO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    image_space_homogenous = np.vstack((image_space_cartesian, np.ones(image_space_cartesian.shape[1])))

    # TO DO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    camera_space_homogenous_normalised = np.matmul(np.linalg.pinv(K), image_space_homogenous)

    # TO DO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    camera_space_cartesian = camera_space_homogenous_normalised[0:2,:]/np.tile(camera_space_homogenous_normalised[2,:],(2,1))
    real_world_cartesian = np.delete(real_world_cartesian, 2, 0)
    homography = calcBestHomography(real_world_cartesian, camera_space_cartesian)    
          
    # TO DO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd

    #I first need to pre-multiply the estimated homography by the inverse of the intrinsic matrix, as indicated in the book
    #WRONG! I already did before. Proceed.

    #Acquire first two columns and compute the SVD of such
    homography_without_last_column = np.delete(homography, -1, axis=1)
    U, L, V_transposed = np.linalg.svd(homography_without_last_column)

    #Acquire first two columns of rotation matrix:
    rotation_first_two_columns = U @ np.array([[1,0],[0,1],[0,0]]) @ V_transposed

    # TO DO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    rotation_third_column = np.cross(rotation_first_two_columns[:,0], rotation_first_two_columns[:,1])
        
    # TO DO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.

    #First assimilate the full rotation matrix
    rotation_matrix = np.hstack((rotation_first_two_columns, rotation_third_column.reshape(-1,1)))

    #Now calculate the determinant and act accordingly
    determinant_rotation_matrix = np.linalg.det(rotation_matrix)
    if determinant_rotation_matrix <= 0:
        #Multiply last column by -1
        rotation_matrix[:,-1] *= -1
    
    # TO DO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    scaling_factor = np.sum(homography_without_last_column / rotation_first_two_columns)/6
    translation = homography[:,-1]/scaling_factor
    
    # TO DO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if translation[-1] < 0:
        translation *= -1
        rotation_matrix[:,0:2] *= -1
 
    # TO DO: Assemble transformation into matrix form
    transformation_matrix = np.vstack((np.hstack((rotation_matrix, translation.reshape(-1,1))), np.array([0., 0., 0., 1.])))
    
    return transformation_matrix

## Now the juicy bit: Estimating an Extrinsics Matrix, T

The problem: We are given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and you've calculated `XImCart` by virtue of the exercise you've completed with camera projection. What we don't have is an extrinsics matrix, `T`. We need to estimate this and you'll need to fill in `estimatePlanePose` and return `TEst`.

Again you can start by negating the noise we add to XImCart to make sure you're on the right track.

In [65]:
# TO DO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
XImCartNoisy = XImCart + np.random.normal(loc=0, scale=1, size=XImCart.shape)#add noise here

# TO DO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function estimatePlanePose
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# If you have got this correct, TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 9.881e-01 -6.726e-02  1.382e-01  5.389e+01]
 [-1.522e-01 -5.533e-01  8.189e-01  8.277e+01]
 [ 2.138e-02 -8.302e-01 -5.570e-01  5.918e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


Once again, above results are acquire with addition of noise. Evidently, the addition of noise has a big impact on small unknowns, such as the value in row 3 column 1. However, without the noise the estimated extrinsic matrix TEst is a really good approximation to the ground truth extrinsic matrix T.